In [1]:
#make sure pymongo works
import pymongo
import pandas as pd
from pprint import pprint

In [2]:
#connecting my pc to mongoDB database
from pymongo import MongoClient
client = MongoClient()

In [12]:
#tokenizing words for text processing
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from stop_words import get_stop_words
from nltk.stem.snowball import SnowballStemmer
from gensim import corpora, models
import gensim

C:\Users\sibbu\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [13]:
import nltk
#nltk.download("stopwords")
import pandas as pd
import json
from pandas.io.json import json_normalize
#import ijson

In [3]:
client = MongoClient('localhost', 27017)

In [5]:
#specifying the database
db = client['Amazon_Reviews']
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [45]:
#specifying the collection to be used in the database
collection = db['All_Reviews']
collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Amazon_Reviews'), 'All_Reviews')

In [46]:
#Testing if able to query from the databse by printing the first row
data = collection.find_one()
data

{'_id': ObjectId('5a332d0e5132162d811c8f3c'),
 'asin': 'B000H00VBQ',
 'helpful': [0, 0],
 'overall': 4.0,
 'reviewText': 'Mysteries are interesting.  The tension between Robson and the tall blond is good but not always believable.  She often seemed uncomfortable.',
 'reviewTime': '10 30, 2013',
 'reviewerID': 'A1RJPIGRSNX4PW',
 'reviewerName': 'J. Kaplan "JJ"',
 'summary': 'Robson Green is mesmerizing',
 'unixReviewTime': 1383091200}

In [83]:
## Fetching summary for different ratings by varying the overall variable value as 1,2,3,4,5
#change the overall value to 1,2,3,4,5 to obtain the summary for those ratings
Review_rating_1 = list(collection.find({'overall':5}, {'summary': 1}))

In [84]:
len(Review_rating_1)

10351164

In [85]:
df1 = pd.DataFrame(Review_rating_1)


In [86]:
#doing text processing, tokenizing, removing stop words, stemming the words 
tokenizer = RegexpTokenizer(r'\w+')
nltk_stpwd = stopwords.words('english')
stop_words_stpwd = get_stop_words('en')
merged_stopwords = list(set(nltk_stpwd + stop_words_stpwd))
sb_stemmer = SnowballStemmer('english')




In [87]:
num_reviews = df1.shape[0]
num_reviews
doc_set = [df1.summary[i] for i in range(num_reviews)]

In [88]:
%%time
texts = []
for doc in doc_set:
    # putting our three steps together
    tokens = tokenizer.tokenize(doc.lower())
    stopped_tokens = [token for token in tokens if not token in merged_stopwords]
    stemmed_tokens = [sb_stemmer.stem(token) for token in stopped_tokens]
    
    # add tokens to list
    texts.append(stemmed_tokens)
print (texts[0])

['robson', 'green', 'great', 'write']
Wall time: 18min 45s


In [89]:
#getting word count and printing top 20 for specified rating
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1
        
sortedwordcount = sorted(frequency.items(), key=lambda k_v: k_v[1], reverse=True) 
sortedwordcount[1:20]


[('love', 793215),
 ('book', 764861),
 ('read', 584972),
 ('good', 508208),
 ('best', 370175),
 ('excel', 297064),
 ('stori', 281664),
 ('one', 250778),
 ('work', 247394),
 ('awesom', 216525),
 ('fun', 197242),
 ('seri', 195162),
 ('perfect', 173108),
 ('wonder', 168843),
 ('amaz', 162717),
 ('anoth', 142461),
 ('must', 140931),
 ('well', 133340),
 ('nice', 132641)]